Import our Asteroid parser to that we can check our example programs.

In [1]:
from asteroid_interp import interp

# Asteroid: The Language

Asteroid is a general purpose programming language heavily influenced by [Python](https://www.python.org), [Lua](http://www.lua.org), and [ML](https://www.smlnj.org).  The language is based around the main design principles that:

1. Simple things should be simple.
2. Everything is a pattern.

Even though the first design principle seems like common sense try writing a "Hello World" program in [Java](https://java.com/en/).  In Asteroid the canonical "Hello World" program is:

>NOTE: The programs on this page are written as strings so we can submit them to the Asteroid *interp* function.

In [2]:
program = \
'''
load "io".
print "Hello World!".
'''

interp(program)

Hello World! 


The second design principle makes use of the fact that computation is pattern manipulation. Consider unsigned integer addition:
```C
unsigned short i = 1
unsigned short j = 2
unsigned short k = i + j
```
We think of this as manipulating values but at the machine level this looks like pattern manipulation:
```C
i --> 00000001
j --> 00000010
k --> 00000011
```
The interesting thing about patterns is that we can manipulate them via a process called *pattern matching*.  Asteroid makes this explicit: everything (by everything we mean all possible expressions) in Asteroid can be interpreted as a pattern/term and therefore manipulated via pattern matching.  

The actual values derived from patterns are usually customary *interpretations* of the patterns.  Consider the bit patterns above.  In order to derive values from these patterns we interpret the patterns as base two digits.  Other interpretations would of course be possible but perhaps not nearly as convenient.  But the fact remains that it is still just an interpretation with the possibility to change it.

Another example of this kind pattern-value duality is the interpretation of "standard" operators.  In the above example we took it for granted that the `+` operator expresses the integer addition. But this is simply based on the *convention* that certain symbols represent certain operations applied to patterns. Again, patterns and values are attached to each other only by convention with the possibility to change this convention.

Compared to other programming languages Asteroid makes this connection between terms and their interpretations explicit by viewing terms as separate entities from the values that they usually represent.  As we will see, by default Asteroid still provides the "standard" interpretations of the conventional patterns but the connection is made explicit and can be changed by the user.

Viewing terms and patterns as entities in their own right divorced from their default interpretation opens up the door to an interesting programming paradigm we would like to call *pattern-level* programming where the program has the ability to directly manipulate the terms representing expressions and also has control over the interpretations of those terms.

To get an idea of how Asteroid accomplishes that, here is a small program that changes the standard interpretation of the `+` operator to multiplication.

In [4]:
program = \
'''
load "standard".  -- load the standard operator interpretations
load "io".        -- load the io system

function funny_add    -- define a function that given two 
    with a, b do      -- parameters a,b will multiply them
        return a * b.
    end function

attach funny_add to __plus__.   -- attach 'funny_add' to '+'
print (3 + 2).                  -- this will print out the value 6
detach from __plus__.           -- restore default interpretation
print (3 + 2).                  -- this will print out the value 5

-- NOTE: '__plus__' is a special symbol representing the '+' operator
'''
interp(program, symtab_dump=False)

6 
5 


## The Basics

In order to illustrate the basics of Asteroid we turn to the canonical factorial program.

In [ ]:
program = \
'''
-- Factorial

load "standard".
load "io".

function fact 
    with 0 do
        return 1
    orwith n do
        return n * fact (n-1).
    end function

print ("The factorial of 3 is: " + fact (3)).
'''

interp(program, exceptions=False, symtab_dump=False)

Most of the program should be pretty self-explanatory. It is written in a typical functional programming style where the first `with`/`orwith` block that unifies with the actual parameters is executed as the body of the function.

Here is another program that recursively walks a list and prints out the elements on the list.  It uses the empty list operator `[ ]` and the head-tail operator `[ | ]`  to pattern-match the input list. The `with` and `orwith` blocks allow different patterns to be applied to the input list.  The blocks are tried in the order they appear in the code.  The first one that matches will execute the commands in its code block.  It is an error if none of the blocks match the input to the function.

In [ ]:
program = \
'''
-- walk a list recursively and print out the elements

load "io".

function print_list 
    with [] do
        ...     -- empty statements, do nothing!
    orwith [h|t] do
        print h.
        print_list t.
    end function

print_list [1,2,3].
'''

interp(program)

Let's try something a little bit more esoteric.  The following program implements [Peano addition](https://en.wikipedia.org/wiki/Peano_axioms#Addition) over the natural numbers.  For this we need the following definitions:

1. 0 is a natural number.
2. For every natural number n, S(n) is a natural number, where S is the successor function.
3. For all natural numbers m and n, m = n if and only if S(m) = S(n).
4. For every natural number n, S(n) = 0 is false. That is, there is no natural number whose successor is 0.

The following program implements Peano addition using pattern matching on the input term. We do not load the standard interpretations for our operators because we are not interested in + behaving as an integer addition at this point.  Here we are only interested in + behaving as a constructor for terms.

In [ ]:
program = \
'''
-- implements Peano addition on terms

-- declare the successor function S as a term constructor so that we 
-- can pattern match on it.
constructor S with arity 1.

-- the 'reduce' function is our reduction engine which recursively pattern matches and
-- rewrites the input term
-- NOTE: during pattern matching free variables are bound to subterms of the original term.
-- For example, the expression S S 0 + 0 is X + 0 will bind X to S S 0 
-- Once a pattern value is bound to a variable it can 
-- be used in the program.  In our case we use the values in the variables to 
-- construct new terms, i.e., S reduce (X + Y)
function reduce
    with X + 0 do                      -- pattern match 'X + 0'
        return reduce X.
    orwith X + S(Y)  do                -- pattern match to 'X + S Y'
        return S(reduce(X + Y)).
    orwith term do                     -- default clause
        return term.
    end function

-- construct a term we want to reduce  
let n = 'S(S(0)) + (S(S(S(0)))).

-- and reduce it!
let rn = reduce n.

-- attach inc behavior/interpretation to the S constructor
load "standard".
load "util".
load "io".

function inc 
    with n do
        return n + 1.
    end function
    
attach inc to S.

-- show that with this behavior both the original term and the rewritten term
-- evaluate to the same value
print ((eval rn) == (eval n)).

'''
interp(program, tree_dump=False, exceptions=False)

Here is another way of doing Peano addition using the built-in 'attach' facility.

In [ ]:
program = \
'''
-- Peano addition using 'attach'

load "standard".
load "io".

-- declare a constructor for our successor function
constructor S with arity 1.

-- declare a function that implements the behavior for the successor function
function inc 
    with n do
        return n + 1.
    end function
    
-- attach the behavior/interpretation to the constructor
attach inc to S.

-- print value using the attached function
print (S (S (S (0))) + S (S (0))).  
'''
interp(program)

Finally, here is the "standard" way of doing this.  Note, however, that the quote operator suspends standard interpretations and the symbols in the quoted expressions are treated as pure constructors.

In [ ]:
program = \
'''
load "standard".
load "util".
load "io".

function S 
    with n do
        return n + 1.
    end function

-- show that the value of the functional term is the same value as the value of the
-- evaluated constructor term.
print (S(S(S(0))) + S(S(0)) == eval('S(S(S(0))) + S(S(0)))). 

'''
interp(program)

## OO Programming in Asteroid

Asteroid's OO model is heavily influenced by Lua's OO model.  At the core the model is a prototype based model.  The interesting thing of course is that Asteroid supports the full pattern/value duality even in the OO model.  That it, Asteroid allows you to pattern match on instatiated objects.  

> NOTE: The self object reference is not fully implemented yet and currently has to be
provided manually as the first parameter to the function.

In [ ]:
program = \
'''
-- demonstrate our OO model in the context of Asteroid's support for pattern/value duality.
load "io".

-- declare a named constructor for a new object type
-- the object has two member slots
constructor Hello with arity 2.

-- instantiate a new object using the constructor. members are name-value pairs so that
-- we can use the dictionary-style dot notation to access members.
let hello_obj = Hello
        (
            ("greeting", "Hello World!"),
            ("print_greeting", lambda with self do print self@{"greeting"})
        ).
 
-- NOTE: 
-- the following statements are all equivalent and access the function stored
-- in the second slot.  all these statements print out "Hello World!"

hello_obj@{"print_greeting"} ().           -- dictionary style reference

hello_obj@[1]@[1] ().                      -- list style reference

with Hello (_, (_,f)) = hello_obj do       -- object pattern matching
    f hello_obj.
end with

-- the following is interesting because it shows that we can pattern match on functions
-- we treat functions as purely syntactic objects -- that is, pattern/value duality is 
-- completely preserved.
let Hello 
    (
        (_, "Hello World!"), 
        (_, lambda with _ do print _)

        -- (_, _)

    )
  = hello_obj.
'''

interp(program, exceptions=True, tree_dump=False)

Note that in the last line of the example above we diverge drastically from the pattern matching available in the standard functional model.  Consider the following ML program:
```ml
val obj = ("hello world!\n", (fn x => print x));
val ("hello world!\n",f) = obj;
f (#1 obj);   
val ("hello world!\n",(fn x => print x)) = obj;
```
The last line in this code example will fail.  That means ML is inconsistent in the way it treats objects: some objects can be inspected using pattern matching and some cannot (functions).This is due to the fact that function constructors in ML are treated as closures, a  mixture of syntax and semantics that cannot be viewed as a pattern. In our approach functions are syntactic objects that can be investigated using pattern matching making them consistent with other constructors/constants and preserve our pattern/value duality.

Here is another OO example.

In [ ]:
program = \
'''
-- another OO example based on the dog example from the Python documentation:
--  https://docs.python.org/3/tutorial/classes.html

load "standard".
load "io".
load "util".

-- Our Dog type constructor
constructor Dog with arity 3.

-- the prototype object
let dog_proto = Dog (
            ("name", ""),
            ("trick", ""),
            ("make_string", 
                lambda with self do return self@{"name"} + " does " + self@{"trick"})
        ).

-- Fido the dog
let fido = copy dog_proto.
let fido@{"name"} = "Fido".
let fido@{"trick"} = "play dead".
print (fido@{"make_string"} ()).

-- Buddy the dog
let buddy = copy dog_proto.
let buddy@{"name"} = "Buddy".
let buddy@{"trick"} = "roll over".
print (buddy@{"make_string"} ()).
'''
interp(program)

## Sorting

In [ ]:
program = \
'''
-- Quicksort

load "standard".
load "io".

function qsort
    with [] do
        return [].
    orwith [a] do
        return [a].
    orwith [pivot|rest] do
        with less=[], more=[] do
            for e in rest do  
                if e < pivot do
                    let less = less + [e].
                else
                    let more = more + [e].
                end if
            end for
                        
            return qsort less + [pivot] + qsort more.
        end with
    end function
    
print (qsort [3,2,1,0])
'''

interp(program, symtab_dump=False)

In [ ]:
program = \
'''
load "standard".
load "util".
load "io".

function bubblesort 
    with list do
        with change do
            repeat
                let change = false.
                for i in 0 to (length list - 2) do
                    if list@[i] > list@[i+1] do
                        -- swap the values
                        let list@[i], list@[i+1] = list@[i+1], list@[i].
                        let change = true.
                    end if
                end for
            until not change.
        end with
        return list. -- return the sorted list
    end function

print (bubblesort [3,2,1,0])


'''

interp(program, exceptions=False)

## Other Random Code Examples

In [ ]:
program =\
'''
-- iterating over compound lists
load "standard".
load "io".

let people = [("Joe",32,"Cook"),("Peter",24,"Pilot"),("Joanne",45,"Doctor")].

for person in people do
    let name, age, occupation = person. 
    print(name + "is" + age + "years old and is a" + occupation). 
end for
'''

interp(program)

In [ ]:
program = \
'''
-- walking a tree defined via tuples of the form (<name>, children*)
load "standard".
load "util".
load "io".

let symtab = [
    ("x", 3),
    ("y", 2),
    ("z", 1)
].

function tree_walk
    with ("+", l, r) do
        return tree_walk l + tree_walk r.
    orwith ("-", l, r) do
        return tree_walk l - tree_walk r.
    orwith ("id", name) do
        return symtab@{name} otherwise 0. -- choice operator if dictionary returns 'none'
    orwith ("num", i) do
        return i.
    orwith node do
        throw Error("unknown node type: " + node).
    end function
    
let tree = ("+", ("num", 2), ("id", "x")).

print(tree_walk tree).
'''

interp(program, exceptions=True)

In [ ]:
program = \
'''
-- implements Peano addition using a lookup table for the rewrite rules

load "standard".
load "util".
load "io".

-- want to use + as a constructor
let add_op = __plus__.  -- save the current interpretation
detach from __plus__. 

-- here we declare the successor function as a term constructor so that we 
-- can pattern match on it.
constructor S with arity 1.

-- each entry in the rule table represents a rewrite rule,
-- first component of the entry is the lhs of the rule
-- second component is the rhs.  The lhs and rhs of
-- the rules are quoted because we want to consider
-- them as pure terms.
let rule_table = 
    [('X + 0, 'reduce X),
     ('X + S(Y), 'S(reduce(X + Y)))].

-- the reduce function is our reduction engine that tries to apply
-- the rules from the rule table to the current input term.
function reduce 
    with term do
        for i in 0 to (length rule_table) - 1 do
            -- limit visibility of free variables of the rewrite rules
            -- to the with block scope
            with lhs, rhs = rule_table@[i] do 
                if term is *lhs do
                    return eval rhs.
                end if
            end with
        end for
        return term.
    end function

let n = 'S(S(0)) + S(S(S(0))). -- peano number 5
let rn = reduce n.

-- check that our reduced rn represents the value 5
function inc 
    with n do
        return n + 1.
    end function
    
attach inc to S.
attach add_op to __plus__.

print ((eval rn) == 5).
'''

interp(program,exceptions=True)